In [19]:
import keras.layers
from keras.utils import plot_model
from keras.models import Model
from keras.layers import BatchNormalization
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D, ZeroPadding2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import Activation, Dropout, Flatten, Dense, Reshape
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import ELU
from keras.layers.recurrent import GRU
import numpy as np
import argparse
import random
import pickle
from keras import regularizers

In [20]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import os
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint
import pandas as pd

In [37]:

xDataRaw = pd.read_csv('xDataMelSmaller.csv' )
genreDataRaw = pd.read_csv( 'genreDataSmaller.csv ))
featDataRaw = pickle.load(open( 'featDataSmaller.csv" ))

featDataRaw = featDataRaw[:,0:4]

xData = np.asarray(xDataRaw) / 255.0
genreData = np.array(genreDataRaw)
featData= np.array(featDataRaw)

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(genreData)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
genres = onehot_encoder.fit_transform(integer_encoded)
print(featData)
print(genreDataRaw.shape)


[[0.417  0.646  0.128  0.0192]
 [0.417  0.646  0.128  0.0192]
 [0.417  0.646  0.128  0.0192]
 ...
 [0.206  0.658  0.033  0.472 ]
 [0.206  0.658  0.033  0.472 ]
 [0.206  0.658  0.033  0.472 ]]
(3750,)


/Users/vinitachaudhary/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [38]:
print(genres)
print(xData.shape)

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
(3750, 128, 128)


In [39]:
def ConvertValenceToLabel(data):
    if data<=.3:
        return 'Sad'
    elif (data>.3 and data<=.6):
        return 'moodNeutral'
    else:
        return 'Happy'
    

In [40]:
danceability = featData[:,0]
energy = featData[:,1]
valence = featData[:,2]
accousticness = featData[:,3]
ValenceData = []
for data in valence:
    ValenceData.extend([ConvertValenceToLabel(data) ])
    


In [41]:
label_encoder_valence = LabelEncoder()
onehot_encoder_valence = OneHotEncoder(sparse=False)
integer_encoded_Valence = label_encoder_valence.fit_transform(ValenceData)
integer_encoded_Valence = integer_encoded_Valence.reshape(len(integer_encoded_Valence), 1)
valenceOneHot = onehot_encoder.fit_transform(integer_encoded_Valence)
print(valenceOneHot)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


/Users/vinitachaudhary/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [42]:
test_portion = .05
dev_portion = .05
test_size = round(len(xData)*test_portion)
dev_size = round(len(xData)*dev_portion)
randTotal= np.random.randint(1,len(xData),test_size+dev_size)
randTest = randTotal[0:test_size]
randDev = randTotal[test_size:]
origIndices= list(range(0, len(xData)))
trainIndices = [x for x in origIndices if x not in randTotal]

print(randDev.shape)
print(genres)

(188,)
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


In [43]:
dev_size

188

In [44]:
devX = xData[randDev]
devGenres =genres[randDev]
devValence =valenceOneHot[randDev]

trainX = xData[trainIndices]
trainGenres =genres[trainIndices]
trainValence =valenceOneHot[trainIndices]

testX = xData[randTest]
testGenres = genres[randTest]
testValence = valenceOneHot[randTest]


testX=testX.reshape(testX.shape[0],testX.shape[1],testX.shape[2],1)
trainX=trainX.reshape(trainX.shape[0],trainX.shape[1],trainX.shape[2],1)
devX = devX.reshape(devX.shape[0],devX.shape[1],devX.shape[2],1)

In [45]:
trainX.shape

(3397, 128, 128, 1)

In [46]:
class Params():

    def __init__(self, json_path):
        self.json_path = json_path

    def save(self, json_path):
        """Saves parameters to json file"""
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)

    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        return self.__dict__

In [55]:
channel_axis = 3
freq_axis = 1
time_axis = 2


def lvl1(width, height, depth,classesGenres, param,activate="softmax"):

    regparam = param.regparam
    dropout = param.dropout
    
    if K.image_dim_ordering() == 'th':
        print ("th")
        channel_axis = 1
        freq_axis = 2
        time_axis = 3
    else:
        print("not th")
        channel_axis = 3
        freq_axis = 1
        time_axis = 2

    if K.image_data_format() =='channels_first':
        input_shape=(1,width,height)
    else:
        input_shape=(width,height,1)
    def convRelu(i, batchNormalization=False):
        nIn = nc if i == 0 else nm[i - 1]
        nOut = nm[i]
        model.add_module('conv{0}'.format(i),nn.Conv2d(nIn, nOut, ks[i], ss[i], ps[i]))
        if batchNormalization:
            model.add_module('batchnorm{0}'.format(i), nn.BatchNorm2d(nOut))
        if leakyRelu:
            model.add_module('relu{0}'.format(i),
                               nn.LeakyReLU(0.2, inplace=True))
        else:
            model.add_module('relu{0}'.format(i), nn.ReLU(True))

     # Input block
    
    model = Sequential()
    model.add(ZeroPadding2D(padding=(0, 40),input_shape= input_shape))
    model.add(BatchNormalization(axis=freq_axis, name='bn_0_freq',input_shape= input_shape))
    # Conv -1 
    model.add(Conv2D(filters=64, kernel_size=3, strides=3, kernel_initializer='glorot_normal',padding='same', name='Conv1', input_shape= input_shape))
    model.add(BatchNormalization(axis=channel_axis, name='bn1'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2),padding='same', name='pool1'))
    model.add(Dropout(0.1, name='dropout1'))
    
     # Conv -2 
    model.add(Conv2D(filters=128, kernel_size=3, strides=3, kernel_initializer='glorot_normal',padding='same', name='Conv2', input_shape= input_shape))
    model.add(BatchNormalization(axis=channel_axis, name='bn2'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(3, 3),strides=(3,3),padding='same', name='pool2'))
    model.add(Dropout(0.1, name='dropout2'))
    
     # Conv -3 
    model.add(Conv2D(filters=128, kernel_size=3, strides=3, kernel_initializer='glorot_normal',padding='same', name='Conv3', input_shape= input_shape))
    model.add(BatchNormalization(axis=channel_axis, name='bn3'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(4, 4),strides=(4,4),padding='same', name='pool3'))
    model.add(Dropout(0.1, name='dropout3'))
    
     # Conv -4 
    model.add(Conv2D(filters=128, kernel_size=3, strides=3, kernel_initializer='glorot_normal',padding='same', name='Conv4', input_shape= input_shape))
    model.add(BatchNormalization(axis=channel_axis, name='bn4'))
    model.add(Activation('elu'))
    model.add(MaxPooling2D(pool_size=(4,4),strides=(4,4), padding='same'))
    model.add(Dropout(0.1, name='dropout4'))
   #model.add(Conv2D(filters=512, kernel_size=2, strides=2, kernel_initializer='glorot_normal',padding='same', name='Conv5', input_shape= input_shape))

    if K.image_dim_ordering() == 'th':
         model.add(Permute((3, 1, 2)))
    model.add(Reshape((1, 128)))
    #convRelu(6,True)
    #model.add(Flatten())
    
  #  GRU block 1, 2, output
    model.add(GRU(32,input_shape=(15,512),return_sequences=True, name='gru1'))
    model.add(GRU(32, return_sequences=False, name='gru2'))
    model.add(Dropout(0.3,name='dropout5'))
   
    model.add(Dense(3, activation='sigmoid', name='output'))
    #model.add(Flatten(name='Flatten'))
    #model.summary()
    
    return model
  
    


In [56]:
'''These are the params I ended up using'''
param = Params('params.json')
param.dict['LR']=.001
param.dict['BatchSize']=150
param.dict['regparam']=.001
param.dict['dropout'] = .5
param.dict['FCLayers']=1
EPOCHS = 75
print(label_encoder_valence.classes_)
print(label_encoder.classes_)

['Happy' 'Sad' 'moodNeutral']
['Electronic' 'Folk' 'Instrumental' 'Pop' 'Rock']


In [57]:
IMAGE_DIMS = trainX[0].shape
H= lvl1(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],depth=IMAGE_DIMS[2], classesGenres = len(label_encoder_valence.classes_),param=param, activate="softmax")

not th


In [58]:
opt = Adam(lr=param.LR)
H.compile(loss= "categorical_crossentropy",loss_weights={'output': 1}, optimizer='adam', metrics=['accuracy'])
# checkpoint
#filepath="weights.best.hdf5"
filepath="weightsCheckpoint.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', 
                             verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]



In [59]:

model_val_gru = H.fit(trainX,trainValence,validation_data=(devX,{"output": devValence}),epochs=EPOCHS,callbacks=callbacks_list, verbose=1)

callbacks=callbacks_list,



Train on 3397 samples, validate on 188 samples
Epoch 1/75
3397/3397 [==============================] - 45s 13ms/step - loss: 1.0048 - acc: 0.4854 - val_loss: 0.9430 - val_acc: 0.5319
Epoch 2/75
3397/3397 [==============================] - 41s 12ms/step - loss: 0.9365 - acc: 0.5378 - val_loss: 0.9559 - val_acc: 0.5638
Epoch 3/75
3397/3397 [==============================] - 41s 12ms/step - loss: 0.8851 - acc: 0.5687 - val_loss: 0.8897 - val_acc: 0.5691
Epoch 4/75
3397/3397 [==============================] - 41s 12ms/step - loss: 0.8627 - acc: 0.5879 - val_loss: 0.8357 - val_acc: 0.6064
Epoch 5/75
3397/3397 [==============================] - 41s 12ms/step - loss: 0.8247 - acc: 0.6155 - val_loss: 0.8961 - val_acc: 0.5426
Epoch 6/75
3397/3397 [==============================] - 43s 13ms/step - loss: 0.8012 - acc: 0.6420 - val_loss: 0.9155 - val_acc: 0.5691
Epoch 7/75
3397/3397 [==============================] - 41s 12ms/step - loss: 0.7720 - acc: 0.6582 - val_loss: 0.8311 - val_acc: 0.6117
E

In [117]:
import matplotlib
import matplotlib.pyplot as plt
import keras
import numpy as np
matplotlib.use('agg')
class TrainingPlot(keras.callbacks.Callback):

    # This function is called when the training begins
    def on_train_begin(self, logs={}):
        # Initialize the lists for holding the logs, losses and accuracies
        self.losses = []
        self.acc = []
        self.val_losses = []
        self.val_acc = []
        self.logs = []

    # This function is called at the end of each epoch
    def on_epoch_end(self, epoch, logs={}):

        # Append the logs, losses and accuracies to the lists
        

In [85]:

import pydotplus
import pydot as pyd
from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pyd
plot_model(H, to_file='model.png')

In [52]:
model_gru_samp_json = H.to_json()
with open("model_gru_samp_json.json", "w") as json_file:
    json_file.write(model_gru_samp_json)
# serialize weights to HDF5
H.save_weights("model_gru_samp_json.h5")
print("Saved model to disk")

Saved model to disk


In [86]:
model_test_val_gru = H.evaluate(testX, [testValence], verbose=1)
H.metrics_names
#H.scores
print("%s: %.2f%%" % (H.metrics_names[1], model_test_val_gru[1]*100))

188/188 [==============================] - 1s 6ms/step
acc: 74.47%


In [53]:
model_test_gru = H.evaluate(testX, [testGenres], verbose=1)
H.metrics_names
#H.scores
print("%s: %.2f%%" % (H.metrics_names[1], model_test_gru[1]*100))

188/188 [==============================] - 1s 5ms/step
acc: 81.38%


In [ ]:
model_gru = H.fit(trainX,trainGenres,validation_data=(devX,{"output": devGenres}),epochs=EPOCHS,callbacks=callbacks_list, verbose=1)
#callbacks=callbacks_list,
# save the model to disk



